In [ ]:
import sys
sys.path.append(sys.path[0] + "/..")

In [ ]:
import json
import os
import properties
import pymysql
import random
import utils

import pandas as pd 

random.seed(10)

In [229]:
cur.close()
conn.close()


### FEVER data preparation

In [230]:
# FEVER Wikipedia DB

mysql_host = os.getenv("HOST", "localhost")
mysql_port = int(os.getenv("PORT", "3306"))
mysql_user = os.getenv("USER", "root")
mysql_pass = os.getenv("PASS", "DimeMP01!")
mysql_db = os.getenv("DB", "fever")

conn = pymysql.connect(host="localhost", port=3306, user="root", password="DimeMP01!", db="fever")
CUR = conn.cursor()

In [231]:
def _get_wiki_evidence(wiki_id: str, sent_idx: int) -> str:
    query = "SELECT * FROM fever.wiki_pages where id='{}'".format(wiki_id)
    CUR.execute(query)
    
    try:
        entry = next(CUR)
        texts = entry[1].split("\n")
        text = texts[sent_idx].split("\t")[1]
        return text
    except Exception as e:
        print("No entry in wiki database for id = {}".format(wiki_id))
    

In [232]:
_get_wiki_evidence("Snakebite_-LRB-album-RRB-", 0)

'Snakebite is the first official release by the British hard rock band Whitesnake .'

In [151]:
def _load_wiki_db(wiki_db_path = "/Users/user/Downloads/wiki-pages/wiki-pages-jsons"):
    wiki_entries = {}
    for file in os.listdir(wiki_db_path):
        with open(os.path.join(wiki_db_path, file)) as f:
            entry = json.load(f)
            wiki_entries[entry['id']] = entry['lines']
            
    return wiki_entries

# wiki_db = _load_wiki_db()
wiki_db = pd.read_csv("/Users/user/Downloads/wiki-pages/wiki-pages.csv")

In [233]:
wiki_db.head()

,Unnamed: 0,id,text,lines
0,0,Snakebite_-LRB-album-RRB-,Snakebite is the first official release by the...,0\tSnakebite is the first official release by ...
1,1,Sin_Sukju,"Sin Suk-ju -LRB- Korean : 신숙주 , hanja : 申叔舟 ; ...","0\tSin Suk-ju -LRB- Korean : 신숙주 , hanja : 申叔舟..."
2,2,"South_Oroville,_California",South Oroville is a census-designated place -L...,0\tSouth Oroville is a census-designated place...
3,3,Southwest_Golf_Classic,The Southwest Golf Classic was a PGA Tour even...,0\tThe Southwest Golf Classic was a PGA Tour e...
4,4,"St._Philip's_Cathedral,_San_Felipe",The St. Philip 's Cathedral -LRB- Catedral de ...,0\tThe St. Philip 's Cathedral -LRB- Catedral ...


In [235]:
entry = wiki_db[wiki_db.id=="Selena"]
entry

,Unnamed: 0,id,text,lines
1839206,1839206,Selena,Selena Quintanilla-Pérez -LRB- -LSB- seˈlena k...,0\tSelena Quintanilla-Pérez -LRB- -LSB- seˈlen...


In [236]:
# load fever training data
fever_train_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/fever/paper_train.jsonl"
train = utils.load_jsonl_file(fever_train_path)

In [237]:
index = 0
train[index]

{'id': 75397,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'evidence': [[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
   [92206, 104971, 'Fox_Broadcasting_Company', 0]]]}

In [238]:
print(len(train[index]['evidence']))
train[index]['evidence']

1


[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [239]:
# filter entries with more than one evidence set
train_multi_evidence = [entry for entry in train if len(entry['evidence'])>1]

In [240]:
print("Initial entries: {}".format(len(train)))
print("Entries with more than one evidence sets: {}".format(len(train_multi_evidence)))


Initial entries: 145449
Entries with more than one evidence sets: 30972


In [265]:
POS_SAMPLE = {
    "claim": "",
    "reference": "",
    "target": "",
    "score": 1,
    "label": None
}
_EVIDENCE_FORMAT = "Title: {}; {}"

def _get_wiki_evidence(wiki_id: str, sentence_idx: int): 
    entry = wiki_db[wiki_db.id==wiki_id]
    sentences = list(entry.to_dict()["lines"].values())[0]
    sentence = sentences.split("\n")[sentence_idx]
    return sentence.split("\t")[1]
    

def _get_evidence_text(evidence_set: list):
    evidence_text = []
    for evidence in evidence_set:
        sentence = _get_wiki_evidence(evidence[2], evidence[3])
        evidence_text.append(_EVIDENCE_FORMAT.format(evidence[2], sentence))
        
    return " ".join(evidence_text)

def _format_claim_evidence(claim, evidence_set):
    return claim + " " + _get_evidence_text(evidence_set)


In [252]:
train_multi_evidence[0]

{'id': 150448,
 'verifiable': 'VERIFIABLE',
 'label': 'SUPPORTS',
 'claim': 'Roman Atwood is a content creator.',
 'evidence': [[[174271, 187498, 'Roman_Atwood', 1]],
  [[174271, 187499, 'Roman_Atwood', 3]]]}

In [263]:
train_multi_evidence[:10]

[{'id': 150448,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'Roman Atwood is a content creator.',
  'evidence': [[[174271, 187498, 'Roman_Atwood', 1]],
   [[174271, 187499, 'Roman_Atwood', 3]]]},
 {'id': 33078,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'The Boston Celtics play their home games at TD Garden.',
  'evidence': [[[49158, 58489, 'Boston_Celtics', 3]],
   [[49159, 58490, 'Boston_Celtics', 3]]]},
 {'id': 6744,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'The Ten Commandments is an epic film.',
  'evidence': [[[23513, 28977, 'The_Ten_Commandments_-LRB-1956_film-RRB-', 0]],
   [[23513, 28978, 'The_Ten_Commandments_-LRB-1956_film-RRB-', 20]]]},
 {'id': 76253,
  'verifiable': 'VERIFIABLE',
  'label': 'SUPPORTS',
  'claim': 'There is a movie called The Hunger Games.',
  'evidence': [[[93100, 106004, 'The_Hunger_Games_-LRB-film-RRB-', 0]],
   [[93100, 106005, 'The_Hunger_Games_-LRB-film-RRB-', 1]],
   [[93100, 106006, 'Th

In [269]:
# prepare training set of POSITIVE samples (similarity score == 1)
bleurt_training_data = []

for entry in train_multi_evidence: 
    try:
        claim = entry['claim']
        targets = []

        for i in range(len(entry["evidence"])-1):
            reference = _format_claim_evidence(entry['claim'], entry["evidence"][i])
            target = _format_claim_evidence(entry['claim'], entry["evidence"][i+1])
            if target != reference:
                new_entry = POS_SAMPLE.copy()
                new_entry['claim'] = entry['claim']
                new_entry['reference'] = reference
                new_entry['target'] = target
                new_entry['label'] = entry['label']
                bleurt_training_data.append(new_entry)
    except Exception as e: 
#         print("Exception for claim: {}".format(claim))
        continue

print("For {} multi evidence entries, {} bleurt training samples created".format(len(train_multi_evidence), 
                                                                                 len(bleurt_training_data)))

KeyboardInterrupt: 

In [274]:
# prepare training set of NEGATIVE samples (similarity score == 1)
bleurt_training_data_neg_samples = []

for entry in bleurt_training_data: 
    claim = entry['claim']
    reference = entry['reference']
    
    while True:
        # randomly select target from another entry which is for a different claim 
        rand_entry = random.sample(bleurt_training_data, 1)[0]
        if rand_entry['claim'] != claim:
            break
    
    new_entry = POS_SAMPLE.copy()
    new_entry['claim'] = claim
    new_entry['reference'] = reference
    new_entry['target'] = claim + " " + " ".join(rand_entry["reference"].split(". ")[1:])
    new_entry['score'] = 0
    bleurt_training_data_neg_samples.append(new_entry)

print("{} negative bleurt training samples created".format(len(bleurt_training_data_neg_samples)))  

9618 negative bleurt training samples created


In [275]:
# Extend bleurt trianing data with negative samples
bleurt_training_data.extend(bleurt_training_data_neg_samples)
print(len(bleurt_training_data))

19236


In [276]:
# save bleurt training data as jsonl file
def to_dict(obj):
    return json.loads(json.dumps(obj, default=lambda o: o.__dict__))

def save_jsonl_file(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            json.dump(to_dict(entry), f)
            f.write("\n")

training_data_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/reference_scorer_training_data"

save_jsonl_file(bleurt_training_data, os.path.join(training_data_path, "fever_train_based.jsonl"))


### Averitec data preparation

In [34]:
# load averitec training data
train_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/averitec/averitec_train.json"
train = utils.load_averitec_base(train_path)

In [35]:
train[0]

AveritecEntry(claim='Hunter Biden had no experience in Ukraine or in the energy sector when he joined the board of Burisma.', label='Supported', justification='No former experience stated.', evidence=[AveritecQA(question='Did Hunter Biden have any experience in the energy sector at the time he joined the board of the  Burisma energy company in 2014', answers=[AveritecAnswer(answer='No', answer_type='Boolean', boolean_explanation="Hunter bidens previous career history does not include work for energy company's.")]), AveritecQA(question='Did Hunter Biden have any experience in Ukraine at the time he joined the board of the  Burisma energy company in 2014', answers=[AveritecAnswer(answer='No', answer_type='Boolean', boolean_explanation="Hunter Bidens previous career history does not include working with Ukrainian company's.")])])

### Synthetic data preparation

In [ ]:
# TODO later 